# IMPORTING STUFFS

In [5]:
import numpy as np

import folium
import pickle
import datetime as dt
import random as rnd
import itertools
import requests
from numpy import mean, std
from scipy.optimize import differential_evolution as optimizer

from math import radians, acos, sin, cos, sin,degrees, sqrt, log, exp


# SOME CONSTANTS

In [6]:
R =  6371.0088e3
kms_per_radian = R # quite obvious :)

model_version = '0_43_2_2'
model_version = '0_99_5'


---
# FUNCTIONS

## SHORTEST ARC
The haversine formula determines the **great-circle distance between two points on a sphere** given their longitudes and latitudes. Important in navigation, it is a special case of a more general formula in spherical trigonometry, the law of haversines, that relates the sides and angles of spherical triangles.

[Haversine funtion](https://en.wikipedia.org/wiki/Haversine_formula)

In [7]:
def geo_distance(P1, P2):
    # [lat, lon]
    lat1, lon1, lat2, lon2 = map(radians, [P1[0], P1[1], P2[0], P2[1]])
    return R * (
        acos(sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(lon1 - lon2))
    )

## SVR DISTANCE MODEL
Our ml_model

In [14]:
from sklearn import svm

svm_model = pickle.load(open(f'models/Little_SVM_{model_version}.pkl', 'rb'))
svm_error_model = pickle.load(open(f'models/Little_SVM_{model_version}_err.pkl', 'rb'))


with open(f'models/country_codes_svm.pkl','rb') as fp:
    country_codes = pickle.load(fp)


def ml_model(t, country_code):
    b=[t]
    b=np.array(b).reshape(1,1)
    v=get_bow(country_code) * t
    v=v.reshape(1,len(country_codes))
    prediction = svm_model.predict(v)
    error = svm_error_model.predict(v) # <----  PORCO DIO E' SBAGLIATO
    D = prediction*20.0e6
    E = error
    return D[0], E[0]


def get_bow(country_code):
    country_dict = {code:0 for code in country_codes}
    country_dict[country_code] = 1.0
    return np.fromiter(country_dict.values(), dtype=float)

---

# DATA

put here you points, country codes, and latency data
(example data of a server in Italy https://dw2xlr3zknsst.cloudfront.net/u)


In [15]:
src_country_codes = ['JP', 'SE', 'US', 'DE', 'US', 'US', 'KR', 'FR', 'IN', 'CA']

points = [[35.68972, 139.69222],
          [59.334591, 18.06324],
          [45.45524, -119.63342],
          [50.110924, 8.682127],
          [39.04237, -77.487244],
          [40.058731, -83.175583],
          [37.5326, 127.024612],
          [48.864716, 2.349014],
          [19.07609, 72.877426],
          [45.50884, -73.58781]
         ]

times = [0.39523700000000006,
         0.18285,
         0.309315,
         0.15809099999999998,
         0.23639700000000002,
         0.26298499999999997,
         0.405799,
         0.168604,
         0.34653599999999996,
         0.25154]


# ESTIMATE OFFSET

In [16]:
%%time

def offset_trilateration(points, times):
    def error(x, p, t):
        err = 0
        for i in range(len(p)):
            d = geo_distance(p[i], x[0:2])
            distance, error = ml_model(t[i] - x[2], src_country_codes[i])
            err +=  abs(d - (distance))
        return err
    
    bounds = [(-50., 80.),
              (-180., 180.),
              (0, min(times)),  #we want non negative latencies
             ]
    
    res = optimizer(error,
                    bounds,
                    args=(points, times),
                    strategy='best1bin',
                    popsize=10,
                    tol=1e-2,
                    init='random')
    
    return res.x, res.fun


              
data = []
k = len(points)

for N in range(k-1, k+1):
    combinations = list(itertools.combinations(range(len(points)), N))
    for pos in combinations:
        _points = [points[index] for index in pos]
        _times  = [times[index] for index in pos]
        
        out, fun = offset_trilateration(_points, _times)
        target = out[0:2]
        print(out[2])
        data.append([target, out[2], fun, pos ])

0.04214883579469814
0.14133589238892663
0.14889259710202746
0.15047822447052292
0.12084712413628013
0.15809099999999998
0.16305390655139496
0.15459190311899718
0.11591693538989113
0.12015690259549015
0.15809099999999998
CPU times: user 1min 44s, sys: 520 ms, total: 1min 45s
Wall time: 1min 45s


In [17]:
data_orig = data
data_orig.sort(key=lambda x: x[2]) # sorting by offset

In [18]:
data = data_orig # first N offsets

In [19]:
offset = data[0][1]
offset = sum([d[1] for d in data]) / len(data) # mean value

print(f'Offeset: {int(offset*1000)}ms')

Offeset: 133ms


In [20]:
times  = [t-offset for t in times] # applying offset

---

# ESTIMATE DISTANCES

In [22]:
dists = []
errs  = []

for i in range(len(points)):
    distance, error = ml_model((times[i]), src_country_codes[i])
    dists.append(abs(distance))
    errs.append(error)
    print(f'{src_country_codes[i]}\t{int((times[i])*1000)} ms\t{int(distance/1000)} Km')
    
errs = list(abs(np.array(errs)))

JP	261 ms	9672 Km
SE	48 ms	2453 Km
US	175 ms	10889 Km
DE	24 ms	805 Km
US	102 ms	6572 Km
US	129 ms	8379 Km
KR	271 ms	8528 Km
FR	34 ms	1521 Km
IN	212 ms	10445 Km
CA	117 ms	6726 Km


In [23]:
#C_Bound!

C = 300.e6/(1.5)/(2**0.5)
min_latency = min(times)
min_latency_index = np.argmin(times)
C_Radius = min_latency/2*C

C_dists = [t/2*C for t in times]


---
# DOING TRILATERATION

In [24]:
%%time
def trilateration(points, distances, errors):
    def error(x, points, distances, errors):
        err = 0
        for i in range(len(points)):
            d = geo_distance(points[i], x[0:2])
            err +=  (d - (distances[i]*x[2] ))**2 / errors[i]
        return err
    
    x0 = np.mean(points, axis=0).tolist()
    bounds = [(-50., 80.),
              (-180., 180.),
              (0.3, 1.7),
              (-10_000e3, +10_000e3)
             ]
    res = optimizer(error, bounds, args=(points, distances, errors), strategy='best1bin', popsize=10, tol=1e-3)
    return res.x, res.fun



data_orig = []

k = len(points)
              
for N in range(k-3,k+1):
    combinations = list(itertools.combinations(range(len(points)), N))
    for pos in combinations:
        tries   = 8
        points_ = [points[index] for index in pos]
        dists_  = [dists[index]  for index in pos]
        errs_   = [errs[index]*dists[index] for index in pos]

        while tries:
            dists__ = [np.random.normal(dists_[d], errs_[d]**0.5) for d in range(len(dists_))]
            out, fun = trilateration(points_, dists__, errs)
            target = out[0:2]

            if geo_distance(target, points[min_latency_index]) :
                data_orig.append([target, abs(1-out[2]), abs(out[3]), fun, pos ])
                tries-=1

CPU times: user 31.6 s, sys: 144 ms, total: 31.8 s
Wall time: 31.8 s


In [25]:
data_orig.sort(key=lambda x: x[3])

In [26]:
data = data_orig[0:int(len(data_orig)/5)] # one tenth of most "precise" data
len(data)

281

# CLUSTERING 

In [29]:
import pandas as pd
import numpy as np

from sklearn.cluster import DBSCAN
from shapely.geometry import MultiPoint


epsilon = 1000.e3 / kms_per_radian

def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    radius = max(list(map(lambda point: geo_distance(point, centroid), cluster )))
    return tuple(centroid), radius


df = pd.DataFrame(data)

db = DBSCAN(eps=epsilon,
            min_samples=1,
            algorithm='ball_tree',
            metric='haversine').fit(np.radians(df[0].tolist()), y=None)

cluster_labels = db.labels_
num_clusters = len(set(cluster_labels))
clusters = pd.Series([df[cluster_labels == n] for n in range(num_clusters)])

print(f'Number of clusters: {num_clusters}')

Number of clusters: 3


# LARGEST CLUSTER

In [30]:
biggest_cluster_index = np.argmax([len(i) for i in clusters])
centermost_points, radius = get_centermost_point(clusters[biggest_cluster_index][0].tolist())

largest_cluster = clusters[biggest_cluster_index][0]

# ERROR ESTIMATION

In [31]:
import math

def error_function(target, points, distances):
    return [abs(geo_distance(points[i], target) - distances[i]) for i in range(len(points))]


residuals =  np.array(error_function(centermost_points, points, dists))**2
residuals /= (np.array(errs))

err = (sum(np.array(residuals))/(sum(1./np.array(errs)**2)))**0.5
print(err/1000)

550.8592746760016


# PLOTTING...

In [32]:
from folium.plugins import HeatMap

world_map = folium.Map(location=centermost_points,
                       zoom_start=2,
                       tiles='cartodbpositron',
                       zoom_control=False,
                       scrollWheelZoom=False,
                       dragging=False,)
                       
#HeatMap(df[0], radius=10, blur=7).add_to(world_map)

HeatMap(largest_cluster, radius=14, blur=9).add_to(world_map)

pos = data[0][3]
for i in range(1,4):
    folium.Circle(centermost_points,
                  color = 'red',
                  radius = err*i, #max(dists)*0.07
                  opacity=1/i,
                 ).add_to(world_map)


for i in range(len(points)):
    folium.Circle(points[i],
               color = 'black',
               radius = C_dists[i],
               opacity=0.05,
              ).add_to(world_map)

    
world_map # plot it!